In [10]:
def format_number(num):
    if num >= 1e12:
        return f"{num/1e12:.0f} T"
    elif num >= 1e9:  # Check if the number is in billions
        return f"{num/1e9:.0f} B"
    elif num >= 1e6:  # Check if the number is in millions
        return f"{num/1e6:.0f} M"
    elif num >= 1e3:  # Check if the number is in thousands
        return f"{num/1e3:.0f} K"
    else:  # If the number is less than 1000
        return str(num)

In [12]:
print(format_number(999))   # Output: 12K
print(format_number(365123456))  # Output: 365M
print(format_number(3651234567891011))  # Output: 365M


999
365 M
3651 T


TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
